# This notebook runs the models for Table 3 (high, med, low usage) with and without using APOE as a covariate

In [ ]:
# Installs
!pip install pandas==1.5.3
!pip install statsmodels
!pip install lifelines==0.26.4

In [ ]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from lifelines import CoxPHFitter

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Select NDDs
ndd_list = ['AD', 'PD', 'DEM']

In [ ]:
# #Load list of codes -- see 04_prep_meds_df for how we got this list
# Note -- not all drugs have enough data to divide into low, med, high usage, but most do
codes = ['diclofenac', 'fluticasone', 'imipramine', 'mirtazapine', 'enalapril', 'terbutaline', 'mirabegron', 'tiotropium', 'temazepam', 'diazepam', 'clomipramine', 'budesonide', 'lisinopril', 'simvastatin', 'celecoxib', 'ciprofloxacin', 'rivastigmine', 'omeprazole', 'dicyclomine', 'etodolac', 'atenolol', 'ranitidine', 'sitagliptin', 'lansoprazole', 'citalopram', 'tamsulosin', 'trazodone', 'prochlorperazine', 'amiodarone', 'trospium', 'escitalopram', 'vardenafil', 'indapamide', 'rabeprazole', 'fludrocortisone', 'gabapentin', 'itraconazole', 'ezetimibe', 'chlordiazepoxide', 'fluvastatin', 'ibuprofen', 'rosiglitazone', 'pioglitazone', 'amphotericin', 'valsartan', 'pravastatin', 'cetirizine', 'salmeterol', 'fluoxetine', 'chlorpromazine', 'risperidone', 'quetiapine', 'tamoxifen', 'oxybutynin', 'tramadol', 'felodipine', 'solifenacin', 'famciclovir', 'paroxetine', 'codeine', 'colchicine', 'fexofenadine', 'erythromycin', 'perindopril', 'digoxin', 'nitrofurantoin', 'tolterodine', 'naproxen', 'spironolactone', 'apixaban', 'clonazepam', 'donepezil', 'oxycodone', 'amantadine', 'methotrexate', 'lactulose', 'amlodipine', 'furosemide', 'carbamazepine', 'metronidazole', 'rasagiline', 'metformin', 'memantine', 'meloxicam', 'pramipexole', 'allopurinol', 'loratadine', 'sumatriptan', 'nifedipine', 'fenofibrate', 'fluconazole', 'buspirone', 'venlafaxine', 'sildenafil', 'dipyridamole', 'loperamide', 'levothyroxine', 'amitriptyline', 'rofecoxib', 'sotalol', 'entacapone', 'lorazepam', 'glimepiride', 'tadalafil', 'bupropion', 'risedronate', 'valproate', 'lamotrigine', 'bumetanide', 'fesoterodine', 'montelukast', 'nortriptyline', 'prednisolone', 'propranolol', 'sulfasalazine', 'insulin', 'indomethacin', 'metoprolol', 'rosuvastatin', 'cimetidine', 'metoclopramide', 'hydroxychloroquine', 'hydroxyzine', 'tetracycline', 'rivaroxaban', 'ramipril', 'verapamil', 'azithromycin', 'galantamine', 'lidocaine', 'diltiazem', 'lithium', 'morphine', 'losartan', 'pantoprazole', 'doxycycline', 'irbesartan', 'levodopa', 'atropine', 'alendronate', 'ofloxacin', 'clopidogrel', 'methocarbamol', 'olanzapine', 'duloxetine', 'eszopiclone', 'warfarin', 'sertraline', 'esomeprazole', 'amoxicillin', 'selegiline', 'finasteride', 'docusate', 'pseudoephedrine', 'aspirin', 'zolpidem', 'ropinirole', 'quinine', 'alfuzosin', 'clonidine', 'doxazosin', 'dutasteride', 'phenytoin', 'candesartan', 'atorvastatin', 'acetaminophen', 'orlistat', 'pregabalin', 'anastrozole', 'levetiracetam', 'bisoprolol', 'baclofen', 'trimethoprim', 'nystatin', 'levocetirizine']
print(len(codes))

In [ ]:
print(ndd_list)
print(len(codes))

In [ ]:
ndd_list = ['AD', 'PD', 'DEM']
timeline = 'low_med_high'
model = 'COX'

results = []

for ndd in ndd_list:
    
    #Load df
    df = pd.read_csv(f'AoU_{ndd}_with_icd10_with_APOE_MAY_12_2025.csv', parse_dates = True, low_memory = False)
    
    # Find codes to use so we don't have to use EVERYTHING
    codes_with_data = []
    lag = 'low'

    
    for code in codes:
        # Choose whether or not to include APOE in analysis here
        m = df[['age_at_tenure', 'SEX', 'tenure', ndd, 'low_' + code, 'med_' + code, 'high_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94']]
        #m = df[['age_at_tenure', 'SEX', 'tenure', ndd, 'low_' + code, 'med_' + code, 'high_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94', 'e3/e4', 'e4/e4']]
        
        n=sum(m[f'{lag}_'+ code])
        df_pair = m[m[f'{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        if n == 0:
            pass
        elif n_pairs < 5:
            pass
        elif n == n_pairs:
            pass
        else:
            print(code)
            codes_with_data.append(code)
    
    print(ndd)
    print(len(codes_with_data))
    
    for code in codes_with_data:

        # Choose whether or not to include APOE in analysis here
        m = df[['age_at_tenure', 'SEX', 'tenure', ndd, 'low_' + code, 'med_' + code, 'high_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94']]
        #m = df[['age_at_tenure', 'SEX', 'tenure', ndd, 'low_' + code, 'med_' + code, 'high_' + code, 'QC0_A04', 'QC0_B02', 'QC0_B37', 'QC0_E10', 'QC0_E11', 'QC0_E27', 'QC0_E78', 'QC0_E87', 'QC0_F20', 'QC0_F31', 'QC0_F32', 'QC0_F33', 'QC0_F40', 'QC0_F41', 'QC0_F42', 'QC0_F43', 'QC0_F44', 'QC0_F45', 'QC0_F48', 'QC0_F50', 'QC0_F51', 'QC0_G40', 'QC0_G43', 'QC0_G47', 'QC0_H66', 'QC0_I10', 'QC0_I11', 'QC0_I12', 'QC0_I15', 'QC0_I20', 'QC0_I21', 'QC0_I25', 'QC0_I47', 'QC0_I48', 'QC0_I49', 'QC0_I50', 'QC0_I60', 'QC0_I61', 'QC0_I62', 'QC0_I63', 'QC0_I65', 'QC0_I66', 'QC0_I67', 'QC0_I69', 'QC0_I82', 'QC0_K04', 'QC0_K05', 'QC0_K20', 'QC0_K21', 'QC0_K22', 'QC0_K25', 'QC0_K31', 'QC0_K51', 'QC0_K59', 'QC0_K70', 'QC0_K71', 'QC0_K72', 'QC0_K73', 'QC0_K74', 'QC0_K75', 'QC0_K76', 'QC0_L25', 'QC0_L40', 'QC0_L50', 'QC0_M06', 'QC0_M13', 'QC0_M15', 'QC0_M16', 'QC0_M17', 'QC0_M18', 'QC0_M19', 'QC0_M32', 'QC0_M45', 'QC0_M79', 'QC0_M80', 'QC0_M81', 'QC0_M88', 'QC0_N04', 'QC0_N10', 'QC0_N18', 'QC0_N19', 'QC0_N30', 'QC0_N31', 'QC0_N32', 'QC0_N39', 'QC0_N40', 'QC0_N94', 'e3/e4', 'e4/e4']]
  
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.001)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        
        results_df = results_df.reset_index()
        test2 = results_df.iloc[2]
        test3 = results_df.iloc[3]
        test4 = results_df.iloc[4]
        

        covariate = code
        
        HR2 = test2['exp(coef)']
        ci_min2 = test2['exp(coef) lower 95%']
        ci_max2 = test2['exp(coef) upper 95%']
        p2 = actual_p[2]
        
        HR3 = test3['exp(coef)']
        ci_min3 = test3['exp(coef) lower 95%']
        ci_max3 = test3['exp(coef) upper 95%']
        p3 = actual_p[3]
        
        HR4 = test4['exp(coef)']
        ci_min4 = test4['exp(coef) lower 95%']
        ci_max4 = test4['exp(coef) upper 95%']
        p4 = actual_p[4]
        
        n2=sum(m[f'low_'+ code])
        df_pair2 = m[m[f'low_'+ code]==1]
        n_pairs2 = sum(df_pair2[ndd])
        
        n3=sum(m[f'med_'+ code])
        df_pair3 = m[m[f'med_'+ code]==1]
        n_pairs3 = sum(df_pair3[ndd])
        #print(n3, n_pairs3)

        n4=sum(m[f'high_'+ code])
        df_pair4 = m[m[f'high_'+ code]==1]
        n_pairs4 = sum(df_pair4[ndd])
        #print(n4, n_pairs4)


        print(covariate, ndd, HR2, ci_min2, ci_max2, p2, n2, n_pairs2)
        print(covariate, ndd, HR3, ci_min3, ci_max3, p3, n3, n_pairs3)
        print(covariate, ndd, HR4, ci_min4, ci_max4, p4, n4, n_pairs4)
        
        results.append((covariate, ndd, model, timeline, 'low', HR2, ci_min2, ci_max2, p2, n_pairs2, n2))
        results.append((covariate, ndd, model, timeline, 'med', HR3, ci_min3, ci_max3, p3, n_pairs3, n3))
        results.append((covariate, ndd, model, timeline, 'high', HR4, ci_min4, ci_max4, p4, n_pairs4, n4))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', 'N_pairs', 'N'))

In [ ]:
#Combine results
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd

In [ ]:
date = 'MAY_14_2025'
output.to_csv(f'AoU_{date}_results_high_low_with_ICD10.csv', header = True, index = False)

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = output   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = f'AoU_{date}_results_high_low_with_ICD10.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/results/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr
